In [ ]:
# импорт библиотек
import pandas as pd
from rdkit import Chem
from chembl_webresource_client.new_client import new_client

In [240]:
# подгузка датасета
activity = new_client.activity

target_chembl_id = 'CHEMBL230'

data = activity.filter(target_chembl_id=target_chembl_id).filter(standard_type='IC50')

df = pd.DataFrame.from_records(data)

print(f'Загружено записей: {len(df)}')

df.to_csv('cox2_raw_data.csv', index=False)
print("Сырые данные сохранены в 'cox2_raw_data.csv'")

Загружено записей: 7980
Сырые данные сохранены в 'cox2_raw_data.csv'


In [241]:
# просмотр датасета
df.head(10)

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,34205,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,0.06
1,None,None,34205,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,0.06
2,None,None,34209,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,3.23
3,None,None,35476,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,0.08
4,None,None,36218,[],CHEMBL769655,Tested in vitro for inhibition against Prostag...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,nM,UO_0000065,None,0.12
5,None,None,36708,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,100.0
6,None,None,39073,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,1.61
7,None,Not Active,39076,[],CHEMBL762916,In vitro inhibitory activity against human Pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,None
8,None,None,40230,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,100.0
9,None,None,40232,[],CHEMBL762912,In vitro inhibitory activity against human pro...,B,None,None,BAO_0000190,...,Homo sapiens,Cyclooxygenase-2,9606,None,None,IC50,uM,UO_0000065,None,100.0


In [ ]:
# просмотр всех колонок
df.columns

Index(['action_type', 'activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type',
       'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value'],
      dtype='object')

In [243]:
# отбор нужных заголовков (думаю, что хорошо было бы добавить колонки 'pchembl_value' (log10 от значения IC50) и 'ligand_efficiency', которые могли бы пригодится при обучении модели, но, к сожалению, они имеют много пустых ячеек (1000+ каждая), поэтому было решено удалить их тоже)
columns_to_keep = ['canonical_smiles', 'standard_value', 'standard_units']
# в колонке standard_units уже приведены к одним единицам измерения (nM), а те, что не поддаются переводу (малая часть), просто удаляются 

df = df[columns_to_keep]
df

,canonical_smiles,standard_value,standard_units
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.0,nM
1,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.0,nM
2,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.0,nM
3,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.0,nM
4,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,nM
...,...,...,...
7975,CC(=O)N/C=C/c1ccc2c(c1)O[C@H](c1ccc(O)c(O)c1)[...,6430.0,nM
7976,Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2...,15.0,nM
7977,O=C1Nc2ccccc2/C1=C/c1coc2ccccc2c1=O,20.0,nM
7978,COc1cc2c(cc1-c1c(C)noc1C)ncc1[nH]c(=O)n([C@H](...,100000.0,nM


In [244]:
# очистка датасета от пропусков и дубликатов
df = df[df['standard_units'] == 'nM']
df = df.dropna(subset=['canonical_smiles'])
df = df.drop_duplicates(subset='canonical_smiles')
df = df[df['standard_value'].notna()]
df

,canonical_smiles,standard_value,standard_units
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.0,nM
2,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.0,nM
3,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.0,nM
4,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,nM
5,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.0,nM
...,...,...,...
7972,CC(=O)NCCc1cc(O)c(O)cc1C(=O)c1ccc(O)c(O)c1,768.0,nM
7974,CC(=O)NCCc1ccc(NC(=O)c2ccc(O)c(O)c2)c(O)c1,6860.0,nM
7975,CC(=O)N/C=C/c1ccc2c(c1)O[C@H](c1ccc(O)c(O)c1)[...,6430.0,nM
7978,COc1cc2c(cc1-c1c(C)noc1C)ncc1[nH]c(=O)n([C@H](...,100000.0,nM


In [245]:
# в целом, последняя колонка не играет роли, поэтому можем просто дронуть ее, но зафиксировать размерность нужно
df = df.rename(columns={'standard_value': 'IC50_nM'})
df = df.drop('standard_units', axis=1)
df

,canonical_smiles,IC50_nM
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.0
2,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.0
3,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.0
4,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12
5,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.0
...,...,...
7972,CC(=O)NCCc1cc(O)c(O)cc1C(=O)c1ccc(O)c(O)c1,768.0
7974,CC(=O)NCCc1ccc(NC(=O)c2ccc(O)c(O)c2)c(O)c1,6860.0
7975,CC(=O)N/C=C/c1ccc2c(c1)O[C@H](c1ccc(O)c(O)c1)[...,6430.0
7978,COc1cc2c(cc1-c1c(C)noc1C)ncc1[nH]c(=O)n([C@H](...,100000.0


In [246]:
# проверка корректности SMILES
def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None


df = df[df['canonical_smiles'].apply(is_valid_smiles)]
print("После проверки валидности SMILES:", df.shape)

После проверки валидности SMILES: (5098, 2)


In [248]:
# все SMILES в датасете валидны, поэтому сохраняем очищенный датасет в новый файл
df.to_csv('cox2_clean_data.csv', index=False)
print("Очищенный датасет сохранён в 'cox2_clean_data.csv'")

Очищенный датасет сохранён в 'cox2_clean_data.csv'
